In [ ]:
import os
import glob
import zipfile
import functools
import re
import time
import shutil

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
#mpl.rcParams['axes.grid'] = False
mpl.rcParams['figure.figsize'] = (12,12)

from sklearn.model_selection import KFold, train_test_split
import matplotlib.image as mpimg
from matplotlib.colors import Normalize
import pandas as pd
import imageio
from PIL import Image

In [ ]:
import tensorflow as tf
import tensorflow.contrib as tfcontrib
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.callbacks import TensorBoard

In [ ]:
# Custom TensorBoard callback taken from https://stackoverflow.com/a/48393723

class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='./logs', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)
        
    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        
        # Pass the non-validation logs to `TensorBoard.on_epoch_end`
        train_logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, train_logs)
        
        val_logs = {k.replace('val_', 'epoch_'): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()

# Preprocessing pipeline

In [ ]:
img_shape = [256, 256, 3]
batch_size = 3
epochs = 100
cv_splits = 1

DEBUG_OUTPUT = False

In [ ]:
def _process_pathnames(fname, label_path):
  """Load the image pair from given paths"""
  # We map this function onto each pathname pair  
  img_str = tf.read_file(fname)
  img = tf.image.decode_png(img_str, channels=3)

  label_img_str = tf.read_file(label_path)
  label_img = tf.image.decode_png(label_img_str, channels=1)
  
  return img, label_img

### Data augmentation

In [ ]:
def _augment(img,
             label_img,
             resize=None, 
             scale=1,
             hue_delta=0,
             contrast_factor_upper_bound = 1,
             saturation_factor_upper_bound = 1,
             horizontal_flip=False,             
             vertical_flip=False,
             rotate90=False):
  """Apply data augmentation"""
  
  # Resize
  if resize is not None:
    img = tf.image.resize_images(img, resize)
    label_img = tf.image.resize_images(label_img, resize)
    
  # Scale
  img = tf.cast(img, tf.dtypes.float32) * scale
  label_img = tf.cast(label_img, tf.dtypes.float32) * scale
  
  # Hue
  if hue_delta > 0:
    img = tf.image.random_hue(img, max_delta=hue_delta)
    
  # Contrast
  if contrast_factor_upper_bound != 1:
    lower_bound = 1 / float(contrast_factor_upper_bound)
    img = tf.image.random_contrast(img, lower=lower_bound, upper=contrast_factor_upper_bound)
    
  # Saturation
  if saturation_factor_upper_bound != 1:
    lower_bound = 1 / float(saturation_factor_upper_bound)
    img = tf.image.random_saturation(img, lower=lower_bound, upper=saturation_factor_upper_bound)
    
  # Ensure images are in [0, 1] range
  img = tf.minimum(tf.maximum(img, 0.0), 1.0)
    
  # Horizontal flip
  if horizontal_flip:
    flip_prob = tf.random_uniform([], 0.0, 1.0)
    img, label_img = tf.cond(tf.less(flip_prob, 0.5),
                                lambda: (tf.image.flip_left_right(img), tf.image.flip_left_right(label_img)),
                                lambda: (img, label_img))
  
  # Vertical flip
  if vertical_flip:
    flip_prob = tf.random_uniform([], 0.0, 1.0)
    img, label_img = tf.cond(tf.less(flip_prob, 0.5),
                                lambda: (tf.image.flip_up_down(img), tf.image.flip_up_down(label_img)),
                                lambda: (img, label_img))
  
  # Random 90° rotations
  if rotate90:
    rotate_count = tf.random_uniform([], 0, 4, dtype=tf.dtypes.int32)
    img = tf.image.rot90(img, rotate_count)
    label_img = tf.image.rot90(label_img, rotate_count)
  
  return img, label_img

In [ ]:
def get_baseline_dataset(filenames, 
                         labels,
                         preproc_fn=functools.partial(_augment),
                         threads=6, 
                         batch_size=batch_size,
                         shuffle=True,
                         repeat=True):           
  num_x = len(filenames)
  # Create a dataset from the filenames and labels
  dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
  dataset = dataset.map(_process_pathnames, num_parallel_calls=threads)
  dataset = dataset.map(preproc_fn, num_parallel_calls=threads)
  
  if shuffle:
    dataset = dataset.shuffle(num_x)
  
  dataset = dataset.batch(batch_size)
  
  # It's necessary to repeat our data for all epochs
  if repeat:
    dataset = dataset.repeat()    
  
  dataset = dataset.prefetch(1)
  
  return dataset

# Define data sets

In [ ]:
img_dir = '../competition-data/training/images'
label_dir = '../competition-data/training/groundtruth'
test_dir = '../competition-data/test'

In [ ]:
train_filenames = os.listdir(img_dir)
x_train_filenames = np.array([os.path.join(img_dir, filename) for filename in train_filenames])
y_train_filenames = np.array([os.path.join(label_dir, filename) for filename in train_filenames])
test_filenames = os.listdir(test_dir)
x_test_filenames = np.array([os.path.join(test_dir, filename) for filename in test_filenames])

print("Number of training examples: {}".format(len(x_train_filenames)))
print("Number of test examples: {}".format(len(x_test_filenames)))

data_splits = []

if cv_splits > 1:
  kFold = KFold(n_splits=cv_splits, shuffle=True)
  data_splits = [(train, val) for train, val in kFold.split(x_train_filenames)]
else:
  train, val = train_test_split(range(len(x_train_filenames)), test_size=0.2)
  data_splits = [(np.array(train), np.array(val))]

### Debug Output: Image paths

In [ ]:
if DEBUG_OUTPUT:
  print("Training images:\n", x_train_filenames[:5])
  print("Training labels:\n", y_train_filenames[:5])

### Debug Output: x_train and y_train images

In [ ]:
if DEBUG_OUTPUT:
  display_num = 2

  r_choices = np.random.choice(len(x_train_filenames), display_num)
  plt.figure(figsize=(10, 10))

  for i in range(0, display_num * 2, 2):
    img_num = r_choices[i // 2]
    x_pathname = x_train_filenames[img_num]
    y_pathname = y_train_filenames[img_num]

    plt.subplot(display_num, 2, i + 1)
    plt.imshow(mpimg.imread(x_pathname), cmap="gray")  
    plt.subplot(display_num, 2, i + 2)
    plt.imshow(mpimg.imread(y_pathname), cmap="gray")

  plt.suptitle("Examples of Images and their Segmentations")
  plt.show()

## Set up train and validation datasets
Note that we apply image augmentation to our training dataset but not our validation dataset. 

In [ ]:
tr_cfg = {
  # Resize input images (and label images)
  'resize': [img_shape[0], img_shape[1]],
  # Adjust value range by a scale factor
  'scale': 1 / 255.,
  # Adjust the hue of an RGB image by random factor choosen in range [-delta, delta]. Default: 0.0
  'hue_delta': 0.1,
  # Random contrast_factor choosen in range [1/upper_bound, upper_bound]. Default: 1.0
  'contrast_factor_upper_bound': 1.33,
  # Random saturation_factor choosen in range [1/upper_bound, upper_bound]. Default: 1.0
  'saturation_factor_upper_bound': 1.33,
  # Apply random left/right flip. Default: False
  'horizontal_flip': True,
  # Apply random up/down flip. Default: False
  'vertical_flip': True,
  # Apply random number of 90 degree rotations. Default: False
  'rotate90': True,
  
}
tr_preprocessing_fn = functools.partial(_augment, **tr_cfg)

In [ ]:
val_cfg = {
  'resize': [img_shape[0], img_shape[1]],
  'scale': 1 / 255.,
}
val_preprocessing_fn = functools.partial(_augment, **val_cfg)

### Debug Output: Preprocessing pipeline

In [ ]:
if DEBUG_OUTPUT:
  temp_ds = get_baseline_dataset(x_train_filenames, 
                               y_train_filenames,
                               preproc_fn=tr_preprocessing_fn,
                               batch_size=1,
                               shuffle=False)
  # Let's examine some of these augmented images
  data_aug_iter = temp_ds.make_one_shot_iterator()
  next_element = data_aug_iter.get_next()
  with tf.Session() as sess: 
    batch_of_imgs, label = sess.run(next_element)

    # Running next element in our graph will produce a batch of images
    plt.figure(figsize=(10, 10))
    img = batch_of_imgs[0]
    label = label[0, :, :, 0]

    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.subplot(1, 2, 2)
    plt.imshow(label, cmap='gray')
    plt.show()

## Define losses
Dice loss is a metric that measures overlap. More info on optimizing for Dice coefficient (our dice loss) can be found in the [paper](http://campar.in.tum.de/pub/milletari2016Vnet/milletari2016Vnet.pdf), where it was introduced. 

We use dice loss here because it performs better at class imbalanced problems by design. In addition, maximizing the dice coefficient and IoU metrics are the actual objectives and goals of our segmentation task. Using cross entropy is more of a proxy which is easier to maximize. Instead, we maximize our objective directly. 

In [ ]:
def dice_coeff(y_true, y_pred):
  smooth = 1.0
  intersection = tf.reduce_sum(y_true * y_pred)
  score = (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)
  return score

def dice_loss(y_true, y_pred):
  loss = 1 - dice_coeff(y_true, y_pred)
  return loss

Here, we'll use a specialized loss function that combines binary cross entropy and our dice loss. This is based on [individuals who competed within this competition obtaining better results empirically](https://www.kaggle.com/c/carvana-image-masking-challenge/discussion/40199). Try out your own custom losses to measure performance (e.g. bce + log(dice_loss), only bce, etc.)!

In [ ]:
def binary_crossentropy(y_true, y_pred):
  return losses.binary_crossentropy(y_true, y_pred)

In [ ]:
def root_mean_squared_error(y_true, y_pred):
  loss = tf.sqrt(losses.mean_squared_error(y_true, y_pred))
  return loss

In [ ]:
# implementation taken from https://lars76.github.io/neural-networks/object-detection/losses-for-segmentation/
# 
def tversky_loss(beta=0.5):
  def loss(y_true, y_pred):
    numerator = tf.reduce_sum(y_true * y_pred)
    false_positives = (1.0 - y_true) * y_pred
    false_negatives = y_true * (1.0 - y_pred)
    denominator = y_true * y_pred + beta * false_positives + (1 - beta) * false_negatives

    return numerator / (tf.reduce_sum(denominator) + tf.keras.backend.epsilon())

  return loss

In [ ]:
def loss_fn(y_true, y_pred):
  # Flatten
  y_true = tf.reshape(y_true, [-1])
  y_pred = tf.reshape(y_pred, [-1])
  
  #loss = binary_crossentropy(y_true, y_pred)
  #loss = dice_loss(y_true, y_pred)
  loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
  #loss = 1/24.0 * binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred) + root_mean_squared_error(y_true, y_pred) + tversky_loss(0.2)(y_true, y_pred)
  
  return loss

# Define our neural network architecture

In [ ]:
def conv_block(input_tensor, num_filters, dilation_rate=1, apply_last_activation=True):
  layer = input_tensor
  layer = layers.Conv2D(num_filters, (3, 3), padding='same', dilation_rate=dilation_rate)(layer)
  layer = layers.Lambda(tf.contrib.layers.layer_norm)(layer)
  layer = layers.Activation('relu')(layer)
  layer = layers.Conv2D(num_filters, (3, 3), padding='same', dilation_rate=dilation_rate)(layer)
  layer = layers.Lambda(tf.contrib.layers.layer_norm)(layer)
  if apply_last_activation:
    layer = layers.Activation('relu')(layer)
  return layer

def residual_block(input_tensor, num_filters, dilation_rate=1):
  # Shape of input_tensor: [batch_size, height, width, num_channels]
  _, _, _, num_channels = input_tensor.shape
  layer = conv_block(input_tensor, num_filters, dilation_rate=dilation_rate)
  # Apply projection if num_filters are not equal
  if num_filters != num_channels:
    input_tensor = layers.Conv2D(num_filters, (1, 1), padding='same')(input_tensor)  
  layer = layers.Add()([input_tensor, layer])
  return layer

def residual_bottleneck_block(input_tensor, num_filters, num_channels=256, dilation_rate=1):
  # Shape of input_tensor: [batch_size, height, width, num_channels]
  layer = layers.Conv2D(num_filters, (1, 1), padding='same')(input_tensor)
  layer = layers.Lambda(tf.contrib.layers.layer_norm)(layer)
  layer = layers.Activation('relu')(layer)
  layer = layers.Conv2D(num_filters, (3, 3), padding='same', dilation_rate=dilation_rate)(layer)
  layer = layers.Lambda(tf.contrib.layers.layer_norm)(layer)
  layer = layers.Activation('relu')(layer)
  layer = layers.Dropout(0.25)(layer)
  # restore num_channels
  layer = layers.Conv2D(num_channels, (1, 1), padding='same')(layer)
  layer = layers.Lambda(tf.contrib.layers.layer_norm)(layer)
  layer = layers.Activation('relu')(layer)  
  layer = layers.Add()([input_tensor, layer])
  return layer

def dilated_spatial_pyramid_pooling(input_tensor, num_filters, num_filters_out=256, dilation_rates=[1]):
  to_concat = []
  to_concat.append(layers.Conv2D(num_filters, (1, 1), padding='same')(input_tensor))
  for d in dilation_rates:
    to_concat.append(layers.Conv2D(num_filters, (3, 3), padding='same', dilation_rate=d)(input_tensor))  
  output = layers.Concatenate(axis=-1)(to_concat)
  output = layers.Conv2D(num_filters_out, (1, 1), padding='same')(output)
  output = layers.Dropout(0.25)(output)
  return output

def encoder_block(input_tensor, num_filters, num_blocks=1):
  layer = input_tensor
  layer = residual_block(layer, num_filters, dilation_rate=1)
  #layer = residual_block(layer, num_filters, dilation_rate=1)
  #layer = residual_block(layer, num_filters, dilation_rate=1)
  pooled = layers.MaxPooling2D((2, 2), strides=(2, 2))(layer)
  #pooled = dilated_spatial_pyramid_pooling(pooled, num_filters, dilation_rates=[1,2,5])
  return pooled, layer

def decoder_block(input_tensor, concat_tensor, num_filters, num_filters_out=256):
  layer = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
  layer = layers.Concatenate(axis=-1)([concat_tensor, layer])
  layer = layers.Lambda(tf.contrib.layers.layer_norm)(layer)
  layer = layers.Activation('relu')(layer)
  layer = residual_block(layer, num_filters, dilation_rate=1)
  #layer = residual_block(layer, num_filters, dilation_rate=1)
  #layer = residual_block(layer, num_filters, dilation_rate=1)
  return layer

In [ ]:
def createInputsOutputs():
  inputs = layers.Input(shape=img_shape)
  # 256
  encoder0_pool, encoder0 = encoder_block(inputs, 32)
  # 128
  encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64)
  # 64
  encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128)
  # 32
  encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256)
  # 16
  encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512)
  # 8
  center = conv_block(encoder4_pool, 1024)
  # center
  decoder4 = decoder_block(center, encoder4, 512)
  # 16
  decoder3 = decoder_block(decoder4, encoder3, 256)
  # 32
  decoder2 = decoder_block(decoder3, encoder2, 128)
  # 64
  decoder1 = decoder_block(decoder2, encoder1, 64)
  # 128
  decoder0 = decoder_block(decoder1, encoder0, 32)
  # 256
  outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(decoder0)
  return inputs, outputs

def createCompiledModel():
  inputs, outputs = createInputsOutputs()
  model = models.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), 
                loss=loss_fn, 
                metrics=[binary_crossentropy, root_mean_squared_error, dice_loss])
  return model

In [ ]:
# Create model path
model_time = time.strftime("%Y-%m-%d_%H-%M-%S")
model_path = 'runs/' + str(model_time) + '/'
os.makedirs(model_path, exist_ok=True)
print('Set model dir to: ' + model_path)

# Copy *.py and *.ipynb files to model_path (save source code)

cwd = os.getcwd()
for file in glob.glob(os.path.join(cwd, '*.py')):
  shutil.copy2(file, model_path)
for file in glob.glob(os.path.join(cwd, '*.ipynb')):
  shutil.copy2(file, model_path)

In [ ]:
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=4, allow_soft_placement=True))
tf.keras.backend.set_session(sess)

model = createCompiledModel()
model.save(model_path + 'model.hdf5')
model.summary()

tf.keras.backend.get_session().close()
del model

# Train model

In [ ]:
histories = []
cv = 0

for trainIndices, valIndices in data_splits:
  cv = cv + 1
  if cv_splits > 1:
    print("Cross validation round {} of {}".format(cv, cv_splits))
  save_model_path = model_path + 'weights_cv' + str(cv) + '_epoch{epoch:03d}_rmse{val_root_mean_squared_error:.4f}.hdf5'
  
  train_ds = get_baseline_dataset(x_train_filenames[trainIndices], 
                                  y_train_filenames[trainIndices],
                                  preproc_fn=tr_preprocessing_fn,
                                  batch_size=batch_size)
  val_ds = get_baseline_dataset(x_train_filenames[valIndices], 
                                y_train_filenames[valIndices],
                                preproc_fn=val_preprocessing_fn,
                                batch_size=batch_size)
  
  sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=4, allow_soft_placement=True))
  tf.keras.backend.set_session(sess)
  
  # Create model
  model = createCompiledModel()
  sess.run(tf.initializers.global_variables())
  # Create callbacks
  tb = TrainValTensorBoard(log_dir=model_path)
  cp = tf.keras.callbacks.ModelCheckpoint(filepath=save_model_path, monitor='val_loss', save_weights_only=True, save_best_only=True, verbose=1)
  es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.005, patience=15, verbose=1, mode='auto')
  # Train model
  history = model.fit(train_ds, 
                    steps_per_epoch=int(np.ceil(len(x_train_filenames[trainIndices]) / float(batch_size))),
                    epochs=epochs,
                    validation_data=val_ds,
                    validation_steps=int(np.ceil(len(x_train_filenames[valIndices]) / float(batch_size))),
                    callbacks=[tb, cp, es])
  histories.append(history)
  
  del model
  tf.keras.backend.get_session().close()

In [ ]:
n = len(histories)
metrics = ['loss', 'root_mean_squared_error', 'dice_loss', 'binary_crossentropy']
metric_count = len(metrics)

plt.figure(figsize=(5*metric_count, 5*n))
for i in range(n):
  for m, metric in enumerate(metrics):
    loss = histories[i].history[metric]
    val_loss = histories[i].history['val_' + metric]
    argmin_val_loss = np.argmin(val_loss)
    epochs_range = range(len(loss))

    plt.subplot(n, metric_count, (i*metric_count)+m+1)
    plt.axvline(argmin_val_loss, color='#ff0000a0')
    plt.plot(epochs_range, loss, label='Training', color='#ff6f42')
    plt.plot(epochs_range, val_loss, label='Validation', color='#4184f3')
    plt.legend(loc='upper right')
    plt.title(metric + ', model' + str(i))

plt.savefig(model_path + 'loss_graph.png', bbox_inches='tight')
plt.show()

# Qualitative evaluation of our performance 

In [ ]:
# Load weights from latest checkpoint
weights_path = sorted(glob.glob(model_path + 'weights*'), reverse=True)[0]
# Alternatively, set weight file directly
#weights_path = "runs/2019-06-21_16-38-37/weights_rmse0.1060_cv1_epoch084.hdf5"

# Create model again and initialize session.
model = createCompiledModel()
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=4, allow_soft_placement=True))
tf.keras.backend.set_session(sess)
sess.run(tf.initializers.global_variables())

model.load_weights(weights_path)
print("Loaded model weights from:", weights_path)

In [ ]:
# Visualize some of the outputs 
num_display=4
colored_masks=True

data_aug_iter = val_ds.make_one_shot_iterator()
next_element = data_aug_iter.get_next()
cols=3

# Running next element in our graph will produce a batch of images
plt.figure(figsize=(5*cols, 5*num_display))
for i in range(num_display):
  batch_of_imgs, label = tf.keras.backend.get_session().run(next_element)
  img = np.array(batch_of_imgs[0])
  label = np.array(label[0, :, :, 0])
  predicted_label = np.array(model.predict(batch_of_imgs)[0, :, :, 0])

  plt.subplot(num_display, cols, cols * i + 1)
  plt.imshow(img)
  plt.title("Input image")
  
  if colored_masks:
    tp = label * predicted_label
    fp = np.abs(tp - predicted_label)
    fn = np.abs(tp - label)
    plt.subplot(num_display, cols, cols * i + 2)
    label = np.stack([fn, tp, 0.0*tp], axis=2)
    plt.imshow(label)
    plt.title("Actual Mask (false negatives in red)")
    plt.subplot(num_display, cols, cols * i + 3)
    predicted_label = np.stack([fp , tp, 0.0*tp], axis=2)
    plt.imshow(predicted_label)
    plt.title("Predicted Mask (false positives in red)")
  
  else:
    plt.subplot(num_display, cols, cols * i + 2)
    plt.imshow(label, cmap="gray")
    plt.title("Actual Mask")
    plt.subplot(num_display, cols, cols * i + 3)
    plt.imshow(predicted_label, cmap="gray")
    plt.title("Predicted Mask")
  
#plt.suptitle("Examples of Input Image, Label, and Prediction")
plt.savefig(model_path + 'validation_examples.png', bbox_inches='tight')
plt.show()

# Create test set predictions

In [ ]:
# create directory for predictions
prediction_dir = weights_path.replace("weights", "predictions", 1).replace(".hdf5", "", 1)
os.mkdir(prediction_dir)

test_ds = get_baseline_dataset(x_test_filenames,
                               x_test_filenames, 
                               preproc_fn=val_preprocessing_fn,
                               batch_size=batch_size,
                               shuffle=False,
                               repeat=False)

data_aug_iter = test_ds.make_one_shot_iterator()
next_element = data_aug_iter.get_next()
count=0

sess = tf.keras.backend.get_session()
try:
  while True:
    batch_of_imgs, label = sess.run(next_element)
    predicted_labels = model.predict(batch_of_imgs)
    
    # rescale images from [0, 1] to [0, 255]
    predicted_labels = predicted_labels / val_cfg['scale']

    for i in range(len(predicted_labels)):
      pred = Image.fromarray(predicted_labels[i, :, :, 0], 'F').resize((608, 608)).convert('L')
      imageio.imwrite(os.path.join(prediction_dir, test_filenames[count]), pred)
      count += 1

except tf.errors.OutOfRangeError:
  pass

In [ ]:
# Show some of the predictions
display_num = 4

r_choices = np.random.choice(len(test_filenames), display_num)
plt.figure(figsize=(10, 5*display_num))

for i in range(0, display_num * 2, 2):
  img_num = r_choices[i // 2]
  x_pathname = x_test_filenames[img_num]
  y_pathname = os.path.join(prediction_dir, test_filenames[img_num])

  plt.subplot(display_num, 2, i + 1)
  plt.imshow(mpimg.imread(x_pathname), cmap="gray")  
  plt.subplot(display_num, 2, i + 2)
  plt.imshow(mpimg.imread(y_pathname), cmap="gray")

#plt.suptitle("Examples of images and their predicted segmentations")
plt.show()

### Mask to submission

In [ ]:
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

# assign a label to a patch
def patch_to_label(patch):
  df = np.mean(patch)
  if df > foreground_threshold:
    return 1
  else:
    return 0


def mask_to_submission_strings(image_filename):
  """Reads a single image and outputs the strings that should go into the submission file"""
  img_number = int(re.search(r"\d+(?=\.png$)", image_filename).group(0))
  im = mpimg.imread(image_filename)
  patch_size = 16
  for j in range(0, im.shape[1], patch_size):
    for i in range(0, im.shape[0], patch_size):
      patch = im[i:i + patch_size, j:j + patch_size]
      label = patch_to_label(patch)
      yield("{:03d}_{}_{},{}".format(img_number, j, i, label))


def masks_to_submission(submission_filename, *image_filenames):
  """Converts images into a submission file"""
  with open(submission_filename, 'w') as f:
    f.write('id,prediction\n')
    for fn in image_filenames[0:]:
      f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(fn))

In [ ]:
submission_filename = weights_path.replace("weights", "submission", 1).replace(".hdf5", ".csv", 1)
image_filenames = [os.path.join(prediction_dir, filename) for filename in os.listdir(prediction_dir)]
masks_to_submission(submission_filename, *image_filenames)

In [ ]:
print("Validating submission file:", submission_filename)
df = pd.read_csv(submission_filename)

print('Shape of csv:', df.shape)
assert df.shape == (135736, 2), "Invalid number of rows or columns in submission file!"
assert df['id'].unique().size == 135736, "Column 'id' should contain 135736 unique values!"

meanPred = df['prediction'].mean()
print("Mean prediction: {:.3f}".format(meanPred))
assert meanPred > 0.05 and meanPred < 0.35, "Very unlikely mean prediction!"

print("Submission file looks OKAY!")